In [ ]:
"""
This file is the entry point for the project
"""
# Begin by importing external libraries using path_setup
# NOTE : Must be ran first, thus PEP8-E402
import path_setup
path_setup.path_setup()
import math  # noqa E402
import sys  # noqa E402
import numpy as np # noqa E402
import datetime as dt # noqa E402
from LatticeDriver import LatticeDriver as lt  # noqs E402
from LatticeDriver import T_to_Beta  # noqa E402
import random as rnd  # noqa E402
# Shebang line for interactive output in vs_code, comment this out if you have troubles running the notebook
import plotly.io as pio
pio.renderers.default = "notebook"

zeroC = 273.15


def rand_time() -> int:
    out = int(dt.datetime.now().strftime('%s'))
    sys.stdout.write(f"Time Seed = {out}\n")
    return(int(dt.datetime.now().strftime('%s')))

def generate_random(gen_num: int) -> list:
    """
        Generates 2 or 3 random numbers whos sum is 100
    """
    if gen_num == 2:
        rand_a = rnd.randint(0, 100)
        rand_b = 100 - rand_a
        return([rand_a, rand_b])
    elif gen_num == 3:
        rand_a = rnd.randint(0, 98)
        if rand_a == 0:
            rand_b = rnd.randint(0, 99)
        else:
            rand_b = rnd.randint(0, 100-rand_a-1)
        rand_c = 100 - rand_a - rand_b
        return([rand_a, rand_b, rand_c])

# Plotting relations
We can look at $\bar{m}$ as a function of the temperature by using the thermodynamic relationship of $\beta = \frac{1}{k_B T}$. Next, we can expand the partition function as so:

$$ Z = \sum_{<i,j>} e^{-\beta E_i} $$
$$ E = -\sum_{<i,j>} J_{<i,j>} \sigma_i \sigma_j,\quad \sigma_i \sigma_j \in \mathbb{Z}$$
$$ \implies Z = a e^{-\beta J},\quad a\in\mathbb{R} $$

From this we can see that the available microstates depend on the $\beta J$ value, and are scaled with the specific spin arrangemet, where $\beta J$ is a dimensionless quantity useful for numerical computation.

## The Heat Capacity of the system

Using the partition function, we can also plot the heat capacity:

$$\langle E \rangle = -\frac{\partial ln(Z)}{\partial \beta}$$

$$\langle E^2 \rangle = \frac{\partial^2 ln(Z)}{\partial \beta^2}$$

With these two values and $\beta = 1/k_bT$, we can write:

$$C_V = \frac{\sigma_E^2}{k_b T^2} $$

$$C_V= (\left<E^2\right>-\left<E\right>^2) \cdot \beta^2 k_b$$

Multiply by the fancy form of one, $J^2/J^2$:

$$C_V= \left(\left<\left(\frac{E}{J}\right)^2\right>-\left<\frac{E}{J}\right>^2 \right) \cdot (\beta J)^2 k_b $$

$$\therefore C_V= \sigma_{E/J}^2 \cdot (\beta J)^{2} k_b$$

From this result, we see that we only need to measure the energy at each evolution step in a MCMC simulation, take the numerical standard deviation, and record the $\beta J$ value used in each simulation to plot the Heat Capacity as a function of a given $\beta J$.

## Magnitization density

Then, we use the above relations to first plot $\bar{m}$ vs $\beta J$ -- where $\bar{m}$ is the average of sum spins per evolution.

# The Detailed Balance Equation
The detailed balance equation (DBE) is a condition used to converge a MCMC simulation to a distribution -- and in this case the thermodynamic Boltzmann distribution. The DBE is a statement that the transition probabilty from one state $\xi$ to some new state $\xi'$ has an equal probability, and can be stated as such:

$$P(\xi\rightarrow\xi') = P(\xi'\rightarrow\xi)$$

The Boltzmann distribution is a distribution that gives a probability that a system will be in a state given the macroenergy of that states collective microstates, and is given by:

$$p(\xi)\propto e^{-E_i\beta}$$

Using this as a cost function of out systems energy and the DBE, we can write:

$$p(\xi) = p(\xi')$$
$$\implies e^{-E_i\beta} = e^{-E_f\beta}$$

Taking the ratio of this, arrive at out cost function for a MCMC evolution to accept or reject a proposed change in the system:

$$\frac{p(\xi)}{p(\xi')} = e^{-\beta(\Delta E)}$$

Therefore for a given choice of $\beta J$, the only deciding factor for accepting or rejecting an evolution is the change in energy of the system from $\xi$ to $\xi'$. We can use the previously derived energy $-\sum_{<i,j>} J_{<i,j>} \sigma_i \sigma_j$, which is the sum neighboring spins times the spin connecting all neighbors.

Using this fact, we can compute the total energy of the system once, then only compute the sum of neighboring spins of a local microstate and multiply this by two times the sign of the proposed spin flip to get the change in energy from $\xi$ to $\xi'$. Using this fact and calling the resulting sum $\Sigma_{nbr}$ and the flip of the initially chosen spin $S_f$, we get:

$$\implies \frac{p(\xi)}{p(\xi')} = e^{-2S_f\Sigma_{nbr}\beta J}$$

# The Metropolis Algorithm
asdf

# The Wolff Algorithm
jkl;

# Setup
This is the setup for the simulation.

# Parameters
## N, M
> The span of the basis vectors

## $\beta J$'s
> The temperature

# Options 0 and 1
> 0 for seeded random or 1 for time based
## Seeded random : 0
> seeds random with 1644121893 by default to generate a repeatable test.

```Probs``` is a list containing 2 -- or 3 integers if voids is enabled, from 0 to 100 such that they all add together to 100.
Each entry in the ```Probs``` list represents the percent chance that random will assign a spin value of 1, -1, or
0 if lattice voids are enabled. You can play with these for interesting results. 
## Time based : 1
> seeds radom with the current epoch time as an integer.

# TODO
It shouldnt be hard to include external magnetic field interactions.

I want to make the $\beta J$ value distance dependant, but need to do more research to impliment this right now.

In [ ]:
N = 64
M = 64
size = [N, M]
total_time = 1000 
J = 42  # eV
print(total_time)
a = T_to_Beta(zeroC)
b = T_to_Beta(1400+zeroC)
print(f'a={a}eV, b={b}eV')
num_points = 10
step = (b-a)/num_points
Beta = np.arange(a, b, step)

lt_c4v_up = lt(1, size, J)
lt_c3v_up = lt(1, size, J, basis=[[1, 0], [0.5, np.sqrt(3)/2]])
lt_c6v_up = lt(1, size, J, [[0.5, np.sqrt(3)/2], [0.5, -np.sqrt(3)/2]])

lt_c4v_dn = lt(1, size, J)
lt_c3v_dn = lt(1, size, J, [[1, 0], [0.5, np.sqrt(3)/2]])
lt_c6v_dn = lt(1, size, J, [[0.5, np.sqrt(3)/2], [0.5, -np.sqrt(3)/2]])

output = input('Enter 0 for seeded random or 1 for time based:')
if output == '0':
    print("option 0 chosen..\n")
    # DOCtest seed = 1644121893
    seed = 1644121893
    lt_c4v_up.randomize(voids=True, probs=[20, 75, 5],
                        rand_seed=seed)
    lt_c3v_up.randomize(voids=False, probs=[25, 75],
                        rand_seed=seed)
    lt_c6v_up.randomize(voids=True, probs=[20, 75, 5],
                        rand_seed=seed)
    lt_c4v_dn.randomize(voids=True, probs=[75, 20, 5],
                        rand_seed=seed)
    lt_c3v_dn.randomize(voids=False, probs=[75, 25],
                        rand_seed=seed)
    lt_c6v_dn.randomize(voids=True, probs=[75, 20, 5],
                        rand_seed=seed)
else:
    print("option 1 chosen.\n")
    print("option 1 chosen.", end='\n')
    output = input('Enable voids (y/n)?')
    voids_enable = True if output == 'y' else False
    rand_n = 2 if voids_enable is False else 3
    seed = rand_time()

    lt_c4v_up.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c3v_up.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c6v_up.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c4v_dn.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c3v_dn.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c6v_dn.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)

# Display
Calling <lattice_object>.display() will display the current spin arangement of the lattice_object.

In [ ]:
lt_c4v_up.plot()
lt_c3v_up.plot()
lt_c6v_up.plot()
lt_c4v_dn.plot()
lt_c3v_dn.plot()
lt_c6v_dn.plot()

# Metropolis Algorithm test
Uncomment if you want to test these. The metropolis algorithm gets ran anyways in the next section.

In [ ]:
# Uncomment the next 4 lines below if you want, but not
# really a reason to as the metropolis algorithm gets
# called anyways from the get_spin_energy function.
# lt_c4v_up.metropolis(total_time, BJ, progress=True,
#                 save=False, auto_plot=True);

In [ ]:
lt_c4v_up.WolffSpinEnergy(Beta, total_time, save=False,
                            auto_plot=True);


In [ ]:
lt_c3v_up.WolffSpinEnergy(Beta, total_time, save=False,
                            auto_plot=True);

In [ ]:
lt_c6v_up.WolffSpinEnergy(Beta, total_time, save=False,
                            auto_plot=True);

In [ ]:
lt_c4v_dn.WolffSpinEnergy(Beta, total_time, save=False,
                            auto_plot=True);

In [ ]:
lt_c3v_dn.WolffSpinEnergy(Beta, total_time, save=False,
                            auto_plot=True);

In [ ]:
lt_c6v_dn.WolffSpinEnergy(Beta, total_time, save=False,
                            auto_plot=True);